In [ ]:
import arcpy
import os

# 设置工作空间和目标数据库
input_folder = r"C:\Users\Runker\Desktop\从江35个占补kml(1)\从江36个占补kml"
output_gdb = r"C:\Users\Runker\Desktop\从江35个占补kml(1)\result.gdb"

# 创建目标数据库（如果不存在）
if not arcpy.Exists(output_gdb):
    arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

# 定义临时工作空间
scratch_folder = arcpy.env.scratchFolder

# 用于存储第一个数据库的坐标系
first_srid = None

# 遍历输入文件夹中的每个KML文件
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith(".kml") or file.endswith(".kmz"):
            kml_file = os.path.join(root, file)
            # 创建唯一临时GDB路径
            temp_gdb = arcpy.CreateUniqueName("temp.gdb", scratch_folder)
            arcpy.CreateFileGDB_management(scratch_folder, os.path.basename(temp_gdb))

            # 转换KML文件的输出文件夹路径
            output_layer_name = os.path.basename(kml_file).replace('.kml', '').replace('.kmz', '')
            output_layer_folder = os.path.join(scratch_folder, output_layer_name + ".gdb")

            # 删除可能存在的同名输出文件夹
            if arcpy.Exists(output_layer_folder):
                arcpy.Delete_management(output_layer_folder)

            # 将KML转换为临时GDB中的要素类
            arcpy.KMLToLayer_conversion(kml_file, scratch_folder, output_layer_name)
            arcpy.env.workspace = output_layer_folder

            # 获取临时GDB中的数据集
            datasets = arcpy.ListDatasets()

            # 遍历每个数据集
            for dataset in datasets:
                arcpy.env.workspace = os.path.join(output_layer_folder, dataset)

                # 获取数据集中的要素类列表
                feature_classes = arcpy.ListFeatureClasses()

                # 按要素类型进行分类并合并到目标GDB
                for fc in feature_classes:
                    shape_type = arcpy.Describe(fc).shapeType

                    # 获取第一个数据库的坐标系
                    if first_srid is None:
                        first_srid = arcpy.Describe(fc).spatialReference

                    # 如果当前要素类的坐标系不同于第一个数据库的坐标系，进行投影
                    if arcpy.Describe(fc).spatialReference.factoryCode != first_srid.factoryCode:
                        projected_fc = os.path.join(output_layer_folder, f"{fc}_projected")
                        arcpy.Project_management(fc, projected_fc, first_srid)
                        fc = projected_fc

                    output_fc_name = f"{shape_type}_{dataset}_{os.path.basename(fc)}"
                    output_fc = os.path.join(output_gdb, output_fc_name)
                    if not arcpy.Exists(output_fc):
                        # 创建新的要素类
                        arcpy.CreateFeatureclass_management(output_gdb, output_fc_name, shape_type, spatial_reference=first_srid)

                    # 合并要素类
                    arcpy.Append_management(fc, output_fc, "NO_TEST")
                    print(f"Features from {kml_file} dataset {dataset} have been merged into {output_fc}")

            # 删除临时GDB
            arcpy.Delete_management(temp_gdb)
print("KML文件转换并合并完成！")